<a href="nyc"><img src = "https://cdn0.tnwcdn.com/wp-content/blogs.dir/1/files/2013/09/nyc.jpg" width = 400> </a>

<h1 align=center><font size = 5>COVID-19 in New York City</font></h1>

## Introduction



## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
<font size = 3>
<li>1.<a href="#item1">Download New York City Datasets</a></li>
<li>2.<a href="#item2">Download Foursquare API Datasets</a></li>
<li>3.<a href="#item3">Analyze Each Neighborhood</a></li>
<li>4.<a href="#item4">Cluster Neighborhoods</a></li>
<li>5.<a href="#item5">Examine Clusters</a>    </li>
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [ ]:
# library to handle data in a vectorized manner
import numpy as np 

# library for data analsysis
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# library to handle JSON files
import json 

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# library to handle requests
import requests 
import warnings
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
import folium 

print('Libraries imported.')

In [ ]:
foursquare_API = False
neighbourhood_data = False

# 1. Download New York City datasets

## 1.0 Load New-York City Zip Codes / Neighborhoods

References :
- https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm
- https://github.com/MacHu-GWU/uszipcode-project

In [ ]:
df_neighborhood = pd.read_excel('zipcodeNYC.xlsx')
df_neighborhood.columns = [ii.lower() for ii in df_neighborhood.columns]
df_neighborhood = df_neighborhood.rename(columns={'zipcode':'zip_code'})
# Strip spaces in zipcode string
df_neighborhood['zip_code']=df_neighborhood['zip_code'].astype('str').str.strip()
print(df_neighborhood.shape)
df_neighborhood.head()

## 1.1 Load New-York City Demographic statistics

Source : http://www.city-data.com/

In [ ]:
#Initialize
df_neighborhood['density']=np.nan
df_neighborhood['median_house_value']=np.nan
df_neighborhood['median_age']=np.nan
df_neighborhood['avg_household_size']=np.nan
df_neighborhood['avg_agi']=np.nan
df_neighborhood['avg_wage']=np.nan

In [ ]:
if neighbourhood_data:
    for ii in range(df_neighborhood.shape[0]):
        
        # Zip Code
        zipcode=str(df_neighborhood['zip_code'].iloc[ii])
        
        
        print('-----')
        print(zipcode)
            
        # Parse HTML data from website 
        data = pd.DataFrame(pd.read_html(r'http://www.city-data.com/zips/'+zipcode+'.html'))
        
        try:
            
            density = int(str(data.iloc[8,0][0]).split('density: ')[1].split(' people')[0].replace(',',''))
            median_house_value = int(str(data.iloc[13,0][1][0]).split('$')[1].replace(',',''))
            median_age = float(str(data.iloc[14,0][1][0]).split(' years')[0])
            avg_household_size = float(str(data.iloc[15,0][1][0]).split(' people')[0])
            avg_agi = int(str(data.iloc[16,0][1][0]).replace(',','').replace('$',''))
            avg_wage = int(str(data.iloc[17,0][1][0]).replace(',','').replace('$',''))

            #Save data in the dataframe
            df_neighborhood['density'].iloc[ii]=density
            df_neighborhood['median_house_value'].iloc[ii]=median_house_value
            df_neighborhood['median_age'].iloc[ii]=median_age
            df_neighborhood['avg_household_size'].iloc[ii]=avg_household_size
            df_neighborhood['avg_agi'].iloc[ii]=avg_agi
            df_neighborhood['avg_wage'].iloc[ii]=avg_wage
            
       
        except :
            print('--ERROR--')
         
    
    # Export Data
    df_neighborhood.to_csv('data_neighborhood.csv',sep=';')

In [ ]:
df_neighborhood = pd.read_csv('data_neighborhood.csv',sep=';', index_col=0)
df_neighborhood['zip_code']=df_neighborhood['zip_code'].astype('str').str.strip()
df_neighborhood.dtypes

In [ ]:
df_neighborhood['total_population']=df_neighborhood['density']*df_neighborhood['landsqmi']
df_neighborhood['total_population']=df_neighborhood['total_population'].astype('float64')
df_neighborhood.head()

In [ ]:
# Drop rows containg blank cells
df_neighborhood=df_neighborhood.dropna(how ='any')
df_neighborhood.shape

## 1.2 Load New-York City COVID-19 cases - by Zip Code

- Source : Github - https://github.com/nychealth/coronavirus-data
- Updated : every day

In [ ]:
url_github_data='https://raw.githubusercontent.com/nychealth/coronavirus-data/master/tests-by-zcta.csv'
# Load data
df_covid19=pd.read_csv(url_github_data, index_col=0).reset_index()
# Rename columns
df_covid19=df_covid19.rename(columns={'MODZCTA':'zip_code','Positive':'nb_positive_covid19'})
# Drop unused columns and null zip codes
df_covid19=df_covid19[['zip_code','nb_positive_covid19']].dropna(subset=['zip_code'])
# Strip spaces in zipcode string
df_covid19['zip_code']=df_covid19['zip_code'].astype('int64').astype('str').str.strip()
print(df_covid19.shape)
df_covid19.head()

#### Merge with COVID-19 dataset, ignoring Neighborhoods without data

In [ ]:
# Merge with demographic data - Inner join
df_data = pd.merge(df_neighborhood,df_covid19,on='zip_code',how='inner')
print(df_data.shape)
df_data.head()

In [ ]:
# Calculate %_pop_positive_covid19
df_data['perc_pop_positive_covid19'] = df_data['nb_positive_covid19']/df_data['total_population']
df_data.head()

In [ ]:
df_data.describe(include='all')

## 1.3 Download New-York City map with Zip Codes

Source : https://geo.nyu.edu/catalog/harvard-tg00nyzcta

In [ ]:
with open('nyzcta-geojson.json') as json_data:
    newyork_data = json.load(json_data)

## 1.4 Vizualize the number of people tested positive for COVID-19

#### Use geopy library to get the latitude and longitude values of New York City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [ ]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

In [ ]:
# create a NYC map
NYC_map = folium.Map(location=[latitude, longitude], zoom_start=10)

In [ ]:
# generate choropleth map using the total immigration of each country to Canada from 1980 to 2013
folium.Choropleth(
    geo_data=newyork_data,
    data=df_data,
    columns=['zip_code', 'perc_pop_positive_covid19'],
    key_on='feature.properties.ZCTA',
    fill_color='YlOrRd', 
    nan_fill_color ='#e7dddd',
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='People tested positive for COVID-19 in New-York City'
).add_to(NYC_map)

# display map
display(NYC_map)

# 2. Download Foursquare API datasets

#### Define Foursquare Credentials and Version

In [ ]:
if foursquare_API :
    CLIENT_ID = 'AKC0LB0R50HQDF5ZJNKXKADAVJDRVJLKSOOOUT3GLWMIVRHM' # your Foursquare ID
    CLIENT_SECRET = '4JYPUHE2U5TZBMO0ULPBP22A53NFHO32MQFSCVKN0AYECMM2' # your Foursquare Secret
    VERSION = '20200301'

    print('Your credentails:')
    print('CLIENT_ID: ' + CLIENT_ID)
    print('CLIENT_SECRET:' + CLIENT_SECRET)

#### Search for specific venue categories - 500m
> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

**Categories** that will be counted :
- **Medical center** :  '4bf58dd8d48988d104941735,'
- **Arts & Entertainment** : '4d4b7104d754a06370d81259'
- **Nightlife spot** : '4d4b7105d754a06376d81259'
- **Outdoors & Recreation** : '4d4b7105d754a06377d81259'
- **Shop & Service** : '4d4b7105d754a06378d81259'
- **Food **:'4d4b7105d754a06374d81259'

In [ ]:
if foursquare_API :
    category_id = ['4bf58dd8d48988d104941735','4d4b7104d754a06370d81259','4d4b7105d754a06376d81259','4d4b7105d754a06377d81259',\
                 '4d4b7105d754a06378d81259','4d4b7105d754a06374d81259']
    category_name =  ['medical_center','arts_entertainment','nightlife_spot','outdoors_recreation','shop_service','food']

    # Initialize
    df_foursquare_API = pd.DataFrame()
    df_foursquare_API=df_data.copy()
    for label in category_name:
        df_foursquare_API[label]=np.nan
    df_foursquare_API.head()

In [ ]:
def get_foursquare_API_data(df, categoryId, categoryName, zipcode):

    # Parameters
    radius = 500
    LIMIT = 100
    # Serach for a Specific Category
    search_query = categoryId
    # Get zip_code latitude/longitude
    latitude = df.loc[zipcode,'latitude']
    longitude = df.loc[zipcode,'longitude']

    # Build the URL
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'\
    .format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

    # Send the GET request
    results = requests.get(url,verify=False).json()

    # assign relevant part of JSON to venues
    venues = results['response']['venues']

    # tranform venues into a dataframe
    dataframe = pd.json_normalize(venues)

    df[categoryName].loc[zipcode] = dataframe.shape[0]
    
    return df

*You need an upgraded Foursquare account to run this function*

In [ ]:
if foursquare_API :
    for ii in range(len(category_id)):
        for jj in range(df_data.shape[0]):
            try :
                print('-----')
                print('ii :',ii,' jj : ',jj)
                df_foursquare_API = get_foursquare_API_data(df_foursquare_API, category_id[ii], category_name[ii], jj)
                if foursquare_API :
                    df_foursquare_API.to_csv('data_nyc.csv',sep=';')
                print(df_foursquare_API.iloc[jj,:])
            except :
                print(ii)
                print(jj)
                break

    df_foursquare_API.head()

# 3. Data Preparation

### Data exploration

In [ ]:
# Load data
df_data = pd.read_csv('data_nyc.csv',sep=';', index_col=0)
df_data.head()

In [ ]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_data['borough'].unique()),
        df_data.shape[0]
    )
)

In [ ]:
df_data.dtypes

### Data preparation

#### Correlation analysis

In [ ]:
df_data[list(df_data.columns)[5:]].corr()

#### Only keep useful columns

In [ ]:
# Only keep useful columns
list_cols = list(df_data.columns)
list_cols = list_cols[0:5]+list_cols[6:10]+[list_cols[11]]+list_cols[14:17]+list_cols[18:20]

# Create dataframe used for machine learning
df_ml = df_data.copy(deep=True)[list_cols]    


# Rename columns
for ii in range(11,len(list_cols)):
    list_cols[ii]='nb_venue_'+list_cols[ii]
df_ml.columns=list_cols
df_ml.head(100)

In [ ]:
import matplotlib.pyplot as plt

x = df_ml['avg_wage']      # year on x-axis
y = df_ml['perc_pop_positive_covid19']     # total on y-axis
fit = np.polyfit(x, y, deg=1)

fit

df_ml.plot(kind='scatter', y='perc_pop_positive_covid19', x='avg_wage', ylim=[0,0.035],figsize=(10, 6), color='darkblue')

plt.title('% of Population tested Positive to COVID-19 versus Average annual wage [$]')
plt.xlabel('Average annual wage [$]')
plt.ylabel('% of Population tested Positive to COVID-19')

# plot line of best fit
plt.plot(x, fit[0] * x + fit[1], color='red') # recall that x is the Years
plt.annotate('y={0:.10f} x + {1:.4f}'.format(fit[0], fit[1]), xy=(150000,0.03))

plt.show()

# print out the line of best fit
#print('No. Immigrants = {0:.0f} * Year + {1:.0f}'.format(fit[0], fit[1]) )

# 3. Cluster Neighbourhoods

### Normalizing over the standard deviation
Now let's normalize the dataset. Normalization is a statistical method that helps mathematical-based algorithms to interpret features with different magnitudes and distributions equally. We use __StandardScaler()__ to normalize our dataset.

In [ ]:
from sklearn.preprocessing import StandardScaler
X = df_ml.values[:,5:]
X = np.nan_to_num(X)
Clus_dataSet = StandardScaler().fit_transform(X)
Clus_dataSet

#### Chose the right number of clusters - Elbow method and Silhouette score

A higher Silhouette Coefficient score relates to a model with better-defined clusters. The Silhouette Coefficient is defined for each sample and is composed of two scores: `

- a: The mean distance between a sample and all other points in the same class.
- b: The mean distance between a sample and all other points in the next nearest cluster.

The Silhouette Coefficient is for a single sample is then given as:
s=b-a/max(a,b)

In [ ]:
from sklearn.metrics import silhouette_score
sse={}
sil_coeff_x=[]
sil_coeff_y=[]

for k in range(1, 10):#
    kmeans = KMeans( init = "k-means++",n_clusters = k, n_init = 20, max_iter = 1000).fit(Clus_dataSet)
    label = kmeans.labels_
    sse[k] = kmeans.inertia_ # Inertia: Sum of distances of samples to their closest cluster center
    if k>=2:
        sil_coeff_y.append(silhouette_score(Clus_dataSet, label, metric='euclidean')) # Silhouette Coefficient 
        sil_coeff_x.append(k)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of cluster")
plt.ylabel("SSE")
plt.title("Elbow Criterion")

In [ ]:
plt.plot(sil_coeff_x, sil_coeff_y)
plt.xlabel("Number of cluster")
plt.ylabel("Silhouette Coefficient")
plt.title("Silhouette Coefficient Criterion")

#### Run k=4

In [ ]:
k_clusters = 4
kmeans = KMeans( init = "k-means++",n_clusters = k_clusters, n_init = 20, max_iter = 1000).fit(X)
df_ml['cluster'] = kmeans.labels_

#### Explore results

In [ ]:
df_ml.groupby('cluster').count()

In [ ]:
df_ml.groupby('cluster').mean()

In [ ]:
df_ml.head()

In [ ]:
# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
rainbow = ['#3554ee','#42ee27','#a96f56','#8c26f6']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_ml['latitude'], df_ml['longitude'], df_ml['neighborhood'], df_ml['cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(NYC_map)
       
NYC_map

# 4. Regression

#### Creating train and test dataset
Train/Test Split involves splitting the dataset into training and testing sets respectively, which are mutually exclusive. After which, you train with the training set and test with the testing set. 
This will provide a more accurate evaluation on out-of-sample accuracy because the testing dataset is not part of the dataset that have been used to train the data. It is more realistic for real world problems.

This means that we know the outcome of each data point in this dataset, making it great to test with! And since this data has not been used to train the model, the model has no knowledge of the outcome of these data points. So, in essence, it’s truly an out-of-sample testing.



In [ ]:
msk = np.random.rand(len(df_ml)) < 0.8
train = df_ml[msk]
test = df_ml[~msk]

#### Feature selection

In [ ]:
independant_variable=list(df_ml.columns)[5:]
independant_variable.remove('perc_pop_positive_covid19')
independant_variable.remove('cluster')
independant_variable.remove('median_age')
#independant_variable.remove('density')
independant_variable

In [ ]:
dependant_variable=['perc_pop_positive_covid19']
dependant_variable

#### Multi-linear Regression

In [ ]:
# write your code here
from sklearn import linear_model
from sklearn.metrics import r2_score

# Initialize linear model
regr = linear_model.LinearRegression(fit_intercept=True)

# Initialize normalized features
x_train = np.asanyarray(train[independant_variable])
x_train_scale = StandardScaler().fit_transform(x_train)

y_train = np.asanyarray(train[dependant_variable])
y_train_scale = StandardScaler().fit_transform(y_train)

x_test = np.asanyarray(test[independant_variable])
x_test_scale = StandardScaler().fit_transform(x_test)

y_test = np.asanyarray(test[dependant_variable])
y_test_scale = StandardScaler().fit_transform(y_test)

# Fit model
regr.fit (x_train_scale, y_train_scale)

# The coefficients
print ('Coefficients: ', regr.coef_)

# Predict test values
y_hat_scale= regr.predict(x_test_scale)
print("Intercept: %.2f" % regr.intercept_ )
print("Residual sum of squares (RSS) : %.2f"
      % np.mean((y_hat_scale - y_test_scale) ** 2))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x_test_scale,y_test_scale ))

print("R2-score: %.2f" % r2_score(y_hat_scale , y_test_scale) )

#### Sigmoid regression

In [ ]:
independant_variable='avg_wage'
dependant_variable='perc_pop_positive_covid19'

In [ ]:
def sigmoid_fkt(a, b, x):
    '''
    Returns array of a horizontal mirrored normalized sigmoid function
    Function parameters a = center; b = width
    '''
    s= 1/(1+np.exp(b*(x-a)))
    return s

In [ ]:
from scipy.optimize import curve_fit
# Initialize linear model
regr = linear_model.LinearRegression()

# Min/Max
max_x = max([max(train[independant_variable]),max(test[independant_variable])])
max_y = max([max(train[dependant_variable]),max(test[dependant_variable])])

# Initialize normalized features
x_train = np.asanyarray(train[independant_variable],dtype=float)/max_x

y_train = np.asanyarray(train[dependant_variable],dtype=float)/max_y

x_test = np.asanyarray(test[independant_variable],dtype=float)/max_x

y_test = np.asanyarray(test[dependant_variable],dtype=float)/max_y


# build the model using train set
popt, pcov = curve_fit(sigmoid_fkt, x_train, y_train, maxfev=5000)

# predict using test set
y_hat = sigmoid_fkt(x_test, *popt)

# evaluation
print("Mean absolute error: %.2f" % np.mean(np.absolute(y_hat - y_test)))
print("Residual sum of squares (MSE): %.2f" % np.mean((y_hat - y_test) ** 2))
print("R2-score: %.2f" % r2_score(y_hat , y_test) )

#Plot
plt.scatter(x_train, y_train,  color='blue')
plt.scatter(x_test, y_test,  color='green')
plt.scatter(x_test, y_hat,  color='red')
XX = np.arange(0,1,0.001)
yy = sigmoid_fkt(XX, *popt)
plt.plot(XX, yy, '-r' )
plt.ylabel('Infected population')
plt.xlabel('Average annual wage')